In [ ]:
import gurobipy as gp
from gurobipy import *


#Number of teams in the competition(must be even)
N=8
#Indexing
I=N
J=N
#Duration in days
T=(2*N)-2 
#Duration of a round in days
Tr= T // 2
#Number of in-between days
Tz=T-1

schedule = Model ()

#All possible combinations of games nxnxt (i == Home game || j = Away game || t = day)
X=schedule.addMVar((I,J,T),vtype=GRB.BINARY, name='Games')
H=schedule.addMVar( (I,Tz), vtype=GRB.BINARY, name='Consecutive Home games ')
A=schedule.addMVar( (J,Tz), vtype=GRB.BINARY, name='Consecutive Away games')


#Combinations Xij where i=j are always equal to zero (a team cannot play against itself)
for i in range(I):
     for j in range(J):
            for t in range(T):
                if i == j:
                    schedule.addConstr(X[i][j][t] == 0)
                    
#Each team plays two times against each other team(once a home game and once an away game)
for i in range(I):
     for j in range(J):
            if i != j:
                schedule.addConstr(sum(X[i][j][t] for t in range(T))== 1)
            
                
#Every day (N/2) games are played
schedule.addConstrs(sum(sum(X[i][j][t] for i in range(I)) for j in range(J)) == (N/2) for t in range(T) )

#Each team plays exactly one game per day
for k in range(N):
    schedule.addConstrs((sum(X[k][j][t] for j in range(J))+sum(X[i][k][t] for i in range(I))) == 1 for t in range(T) )

#Every pair of teams meet on the same nth day of each round
for i in range(I):
     for j in range(J):
            for t in range(Tr):
                if i != j:
                    schedule.addConstr(X[i][j][t] == X[j][i][t+Tr])
                    schedule.addConstr(X[j][i][t] == X[i][j][t+Tr])


#Decision variables H und A are determined through the following conditions
#H (respectively A) is assigned the value 1 on day t, when a team plays Home games(r. Away games) on t and t+1.



for t in range(Tz):
    schedule.addConstrs(sum(X[i][j][t] + X[i][j][t+1] for j in range(J)) <= (H[i][t] + 1) for i in range(I))
    schedule.addConstrs(sum(X[i][j][t] + X[i][j][t+1] for i in range(I)) <= (A[j][t] + 1) for j in range(J))
    
               
#The sum of consecutive home games
sumhome=sum(sum(H[i][t] for t in range(Tz)) for i in range(I) )

#The sum of consecutive away games
sumaway=sum(sum(A[j][t] for t in range(Tz)) for j in range(J) )

#Objective
Obj=sumhome+sumaway

schedule.setObjective(Obj, GRB.MINIMIZE)
schedule.setParam('TimeLimit', 200)   

schedule.optimize()
